<h1>Pretrained Convnets</h1><br><br>
Es ist möglich die Convolutional Base, also die ersten Conv und Max-Pooling-Schichten, bereits tranierter Netze zu nutzen. Auch wenn diese auf anderen Problemen traniert worden sind, dienen diese Schichten als Modell der visuellen Welt und können sich so als nützlich erweisen. Man kann vortrainerte Netze auf zwei Weisen verwenden: Feature Extraction und Fine-Tuning.<br><br>
<b>1. Feature Extraction</b><br>
Hier wird die alte Conv-Basis genutzt. Das bedeutet, dass die Convolution- und Max-Pooling Schichten anderer Netzwerke verwendet werden. Auf diesen wird ein neuer Klassifizierer aus Dense-Schichten gesetzt, der deinem Problem angepasst ist. Je "weiter unten" die Schichten sind, die man verwendet, desto generischer sind die gelernten Muster (unten z.B. "schräger Strich", oben "Konzept Ohr") und desto eher sind diese auf dein Problem anwendbar. Auch ist es gut, die Convbasis ähnlicher Probleme zu verwenden zu verwenden. So wird eine Basis, die urspünglich auf die Klassifizierung von vielen Tieren trainiert wurde auch für die binäre Klassifizierung von "Hund/kein Hund" nützlich sein, da in der Basis bereits das Konzept des Hundes zu finden ist.<br><br>
Keras bringt bereits einige Convbasen mit:

In [1]:
# diese sind in keras.applications zu finden
# das ImageNet-Problem enthielt mehrere Katzen- und Hundeklassen
# wir nehmen die Basis des VGG16-Netzwerks

from keras.applications import VGG16
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False, # soll der Klassifizierer mit einbezogen werden?
    input_shape = (150, 150, 3) # Wie große sind die Bilder? (height, width, channels (rgb))
)
conv_base.summary()

layers = [layer.input for layer in conv_base.layers]
for layer in layers:
    print(layer, '\n')


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

Jetzt, wo die Basis importiert worden ist, kann man auf zwei verschiednen Arten weitermachen:<br>
1. Du kannst einmal alle Bilder durch die Basis laufen lassen und die Numpy-Arrays in der Form (4, 4, 512) aufzeichen und diese später zum Trainieren des Klassifizierers nutzen. Vorteile: schnell, Nachteile: keine Data-Augmentation möglich, also wird schneller Overfitting eintreten.
2. Zum anderen kannst du deinen Klassifizierer gleich auf die Basis setzen, die Basis einfrieren und dann diese beiden Teile zusammen trainieren. Vorteile: Data Augmentation möglich, Nachteil: sehr langsam, da rechn. teuer.


In dem hier vorliegenden Problem sollte eigentlich die zweite Variante genutzt werden, da bei CNNs mit kleinem Datensatz Data Augmentation essentiell ist.<br><br>
<b>1.1. Seperate Nutzung der beiden Teile</b><br>
Als erstes wirst du wieder dem ImageGenerator die Bilder als NumPy-Arrays entziehen und dann das Ergebnis der Basis mittels der <code>predict()</code>-Methode aufzeichnen und später als Datensetz zum Trainieren des Klassifizieres verwenden.

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = "/home/dominik/Documents/Datasets/cats_and_doggos/small"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

data_gen = ImageDataGenerator(rescale = 1.0 / 255.0)
batch_size = 20

# Gets you the (features, labels), feautures are predicted by the base from the pics.
def extract_features(directory, sample_count):
    features = np.zeros(shape = (sample_count, 4, 4, 512)) # shape of output of base
    labels = np.zeros(shape = sample_count) # label vector as long as sample
    
    # will return images from directory as generator
generator = data_gen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = batch_size,
    class_mode = "binary"
)
    
for inputs_batch, labels_batch in generator:
    print(labels_batch) # HIER PROBLEM, KANN LABEL NICHT FINDEN
i = 0
    #for inputs_batch, labels_batch in generator:
    #    features_batch = conv_base.predict(inputs_batch, steps = batch_size) # takes the predictions of the base
    #    features[i * batch_size : (i + 1) * batch_size] = features_batch
    #    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    #    i += 1
    #    if i * batch_size >= sample_count:
    #        break
    #return features, labels

#train_features, train_labels = extract_features(train_dir, 2000)
#validation_features, validation_labels = extract_features(validation_dir, 1000)
#test_features, test_labels = extract_features(test_dir, 1000)
    
# ... S. 22/47

<b>1.2 Feature Extraction mit Data Augmentation</b><br>
Hier werden die dicht vernetzten Schichten auf die Convolution-Basis aufgesetzt, was wesentlich teurer und langsamer ist. Dazu müssen die Convolution-Schichten eingefroren werden, da sonst die vorher gelernten Muster verändert werden. Da die Gewichte der <code>Dense</code>-Schichten  zufällig initialisiert sind, werden große Updates durch das Netzwerk propagieren und so die gelernten Muster zerstören.

In [1]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

# Einfrieren der conv_base
conv_base.trainable = False

model.summary()

# Hier den Code von S. 25

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


NameError: name 'conv_base' is not defined